# This modeling uses NLP to predict user ratings  from user's kindle book 

# review




# 1. Data Loading and data preprocessing

In [1]:
#Dependencies
import numpy as np
import pandas as pd
from pathlib import Path
from pycaret.nlp import *
#from pycaret.classification import *
from pycaret.regression import *
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer


In [3]:
df = pd.read_csv('data.csv')

df.head(5)

,actual_ratings,reviewText,summary,verified,vote
0,4,"pretty good story, a little exaggerated, but I...",pretty good story,True,0
1,5,"If you've read other max brand westerns, you k...",A very good book,True,0
2,5,"Love Max, always a fun twist",Five Stars,True,0
3,5,"As usual for him, a good book",a good,True,0
4,5,MB is one of the original western writers and ...,A Western,True,2


In [98]:
#df.rename({'overall': 'actual_ratings'}, axis=1, inplace=True)

In [ ]:
#Dropping unwanted columns
#new_df.drop(['reviewerID','asin','vote','newReviewTime'], axis=1, inplace=True)

In [ ]:
#new_df.drop(['Unnamed: 0'], axis=1, inplace=True)

In [4]:
#Removing common stopwords
stop = stopwords.words('english')

In [5]:
#Removing common stopwords
df['reviewText_sws'] = df['reviewText'].apply(lambda x: ' '.join([word for word in str(x).split() if word not in (stop)]))

In [96]:
#Drop duplicate columns
#df.drop_duplicates(inplace=True, keep='first')

In [6]:
df.head(3)

,actual_ratings,reviewText,summary,verified,vote,reviewText_sws
0,4,"pretty good story, a little exaggerated, but I...",pretty good story,True,0,"pretty good story, little exaggerated, I liked..."
1,5,"If you've read other max brand westerns, you k...",A very good book,True,0,"If read max brand westerns, know expect, not, ..."
2,5,"Love Max, always a fun twist",Five Stars,True,0,"Love Max, always fun twist"


In [7]:
#checking for null values in the columns
df.isnull().sum()

actual_ratings    0
reviewText        0
summary           0
verified          0
vote              0
reviewText_sws    0
dtype: int64

## Bag Of Words
## Bag Of Words (BOW) is a popular algorithm for representing text in numbers. It relies on the frequency of the words in the document. BOW has numerous applications like document classification, topic modeling, and text similarity. In BOW, every document is represented as the frequency of words present in the document. So, the frequency of words represents the importance of the words in the document.
## CountVectorizer
### Convert a collection of text documents to a matrix of token counts.
### implements both tokenization and occurrence counting in a single class:
### This implementation produces a sparse representation of the counts using scipy.sparse.csr_matrix.

In [8]:
#Convert a collection of text documents to a matrix of token counts.
#implements both tokenization and occurrence counting in a single class:
tf_vectorizer = CountVectorizer(max_features=800,stop_words={'english'}, ngram_range=(1,1))

In [9]:
features = tf_vectorizer.fit_transform(df['reviewText_sws'].apply(lambda features : np.str_(features)))

In [10]:
features_df = pd.DataFrame(features.toarray(), columns=tf_vectorizer.get_feature_names())

In [ ]:
#print("Vocabulary: ", tf_vectorizer.vocabulary_)

In [11]:
features_df.head(5)

,19th,20th,abagnale,able,about,absolutely,academy,account,action,actual,actually,add,adventure,after,again,age,ago,all,allison,almost,alone,along,already,also,although,always,amazing,amazon,american,amount,and,anita,another,anyone,anything,are,argeneau,around,art,articles,as,at,attention,author,authors,available,away,back,bad,bank,based,battle,be,became,because,become,becomes,bed,before,begin,beginning,begins,behind,believable,believe,bennie,best,better,big,birthday,bit,bite,black,bliss,blood,boba,book,books,boring,bought,bourgeoisie,brennan,brief,bring,business,but,buy,by,call,came,can,cannot,capital,capitalism,capitalist,care,carnton,carrie,case,catch,cause,century,certainly,chance,chandler,change,chapter,chapters,character,characters,child,children,civil,class,classes,clear,clearly,cm_cr_arp_d_rvw_txt,collection,college,come,comes,coming,communism,communist,compelling,complete,completely,complex,con,concept,content,continue,control,copy,could,countries,country,couple,course,cover,created,crime,critique,culture,danger,dark,data,daughter,day,days,dead,deal,death,decided,definitely,descriptions,despite,detail,details,detective,developed,development,dialogue,did,diet,different,difficult,disappointed,division,do,doctor,done,down,dp,dr,due,earlier,early,easily,easy,economic,economics,edition,either,else,emily,end,ending,engels,engine,enjoy,enjoyable,enjoyed,enough,entertaining,entire,equal,era,especially,essay,etc,etna,european,even,events,eventually,ever,every,everyone,everything,evil,evolution,exactly,example,excellent,except,exciting,existence,existing,expect,expected,experience,face,fact,fall,family,fan,far,fast,father,favorite,fbi,feel,feeling,feels,felt,female,fett,fiction,figure,finally,find,finds,finish,finished,fire,first,five,follow,...,published,put,quality,question,questions,quick,quickly,quite,rate,rates,rather,rating,re,read,reader,readers,reading,real,reality,really,reason,reasons,recommend,recommended,ref,relations,relationship,rest,return,review,reviews,revolution,rich,right,rights,romance,romantic,root,rosato,said,sands,save,saved,say,saying,says,scenes,scottoline,second,section,see,seem,seemed,seems,seen,self,sense,series,set,several,sex,she,ship,short,shreve,side,sight,simple,simply,since,sister,slave,slow,small,so,social,society,some,someone,something,sometimes,somewhat,soon,sort,source,south,sowell,space,speak,spoilers,stand,star,stars,start,started,starts,state,states,steam,still,stop,stories,story,straight,strong,struggle,style,subject,subscription,sure,suspense,system,take,takes,taking,tale,tassel,tell,tells,text,thanks,that,the,them,then,theory,there,these,they,thing,things,think,thinking,this,thomas,though,thought,thoughts,three,throughout,thus,tied,time,times,to,today,together,told,tony,too,took,town,trading,travel,trial,tribe,tried,trilogy,true,truly,try,trying,turn,turns,twist,two,type,ultimately,understand,understanding,unfortunately,unique,unlike,up,upon,us,use,used,usually,utf8,value,vampire,vampires,van,various,vatta,ve,version,very,viable,view,villain,volume,vong,wait,want,wanted,wants,war,wars,was,washington,way,ways,we,wealth,weight,well,went,what,when,whether,which,while,who,whole,whose,why,wife,will,wish,with,within,without,woman,women,wonderful,word,words,work,workers,working,works,world,worth,would,write,writer,writers,writes,writing,written,wrong,wrote,year,years,yes,yet,you,young
0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [12]:
new_df = pd.concat([features_df,df['actual_ratings']],axis=1)

In [106]:
#new_df=features_df
#extracted_col = df["actual_ratings"]
#new_df = new_df.join(extracted_col)

In [13]:
new_df

,19th,20th,abagnale,able,about,absolutely,academy,account,action,actual,actually,add,adventure,after,again,age,ago,all,allison,almost,alone,along,already,also,although,always,amazing,amazon,american,amount,and,anita,another,anyone,anything,are,argeneau,around,art,articles,as,at,attention,author,authors,available,away,back,bad,bank,based,battle,be,became,because,become,becomes,bed,before,begin,beginning,begins,behind,believable,believe,bennie,best,better,big,birthday,bit,bite,black,bliss,blood,boba,book,books,boring,bought,bourgeoisie,brennan,brief,bring,business,but,buy,by,call,came,can,cannot,capital,capitalism,capitalist,care,carnton,carrie,case,catch,cause,century,certainly,chance,chandler,change,chapter,chapters,character,characters,child,children,civil,class,classes,clear,clearly,cm_cr_arp_d_rvw_txt,collection,college,come,comes,coming,communism,communist,compelling,complete,completely,complex,con,concept,content,continue,control,copy,could,countries,country,couple,course,cover,created,crime,critique,culture,danger,dark,data,daughter,day,days,dead,deal,death,decided,definitely,descriptions,despite,detail,details,detective,developed,development,dialogue,did,diet,different,difficult,disappointed,division,do,doctor,done,down,dp,dr,due,earlier,early,easily,easy,economic,economics,edition,either,else,emily,end,ending,engels,engine,enjoy,enjoyable,enjoyed,enough,entertaining,entire,equal,era,especially,essay,etc,etna,european,even,events,eventually,ever,every,everyone,everything,evil,evolution,exactly,example,excellent,except,exciting,existence,existing,expect,expected,experience,face,fact,fall,family,fan,far,fast,father,favorite,fbi,feel,feeling,feels,felt,female,fett,fiction,figure,finally,find,finds,finish,finished,fire,first,five,follow,...,put,quality,question,questions,quick,quickly,quite,rate,rates,rather,rating,re,read,reader,readers,reading,real,reality,really,reason,reasons,recommend,recommended,ref,relations,relationship,rest,return,review,reviews,revolution,rich,right,rights,romance,romantic,root,rosato,said,sands,save,saved,say,saying,says,scenes,scottoline,second,section,see,seem,seemed,seems,seen,self,sense,series,set,several,sex,she,ship,short,shreve,side,sight,simple,simply,since,sister,slave,slow,small,so,social,society,some,someone,something,sometimes,somewhat,soon,sort,source,south,sowell,space,speak,spoilers,stand,star,stars,start,started,starts,state,states,steam,still,stop,stories,story,straight,strong,struggle,style,subject,subscription,sure,suspense,system,take,takes,taking,tale,tassel,tell,tells,text,thanks,that,the,them,then,theory,there,these,they,thing,things,think,thinking,this,thomas,though,thought,thoughts,three,throughout,thus,tied,time,times,to,today,together,told,tony,too,took,town,trading,travel,trial,tribe,tried,trilogy,true,truly,try,trying,turn,turns,twist,two,type,ultimately,understand,understanding,unfortunately,unique,unlike,up,upon,us,use,used,usually,utf8,value,vampire,vampires,van,various,vatta,ve,version,very,viable,view,villain,volume,vong,wait,want,wanted,wants,war,wars,was,washington,way,ways,we,wealth,weight,well,went,what,when,whether,which,while,who,whole,whose,why,wife,will,wish,with,within,without,woman,women,wonderful,word,words,work,workers,working,works,world,worth,would,write,writer,writers,writes,writing,written,wrong,wrote,year,years,yes,yet,you,young,actual_ratings
0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [14]:
new_shuffle_df = new_df.sample(frac=1)

# Define a size for your train set 
new_train_size = int(0.75 * len(df))

# Split your dataset 
new_train_df = new_shuffle_df[:new_train_size]
new_test_df  = new_shuffle_df[new_train_size:]

In [18]:
new_train_df.head(3)

,19th,20th,abagnale,able,about,absolutely,academy,account,action,actual,actually,add,adventure,after,again,age,ago,all,allison,almost,alone,along,already,also,although,always,amazing,amazon,american,amount,and,anita,another,anyone,anything,are,argeneau,around,art,articles,as,at,attention,author,authors,available,away,back,bad,bank,based,battle,be,became,because,become,becomes,bed,before,begin,beginning,begins,behind,believable,believe,bennie,best,better,big,birthday,bit,bite,black,bliss,blood,boba,book,books,boring,bought,bourgeoisie,brennan,brief,bring,business,but,buy,by,call,came,can,cannot,capital,capitalism,capitalist,care,carnton,carrie,case,catch,cause,century,certainly,chance,chandler,change,chapter,chapters,character,characters,child,children,civil,class,classes,clear,clearly,cm_cr_arp_d_rvw_txt,collection,college,come,comes,coming,communism,communist,compelling,complete,completely,complex,con,concept,content,continue,control,copy,could,countries,country,couple,course,cover,created,crime,critique,culture,danger,dark,data,daughter,day,days,dead,deal,death,decided,definitely,descriptions,despite,detail,details,detective,developed,development,dialogue,did,diet,different,difficult,disappointed,division,do,doctor,done,down,dp,dr,due,earlier,early,easily,easy,economic,economics,edition,either,else,emily,end,ending,engels,engine,enjoy,enjoyable,enjoyed,enough,entertaining,entire,equal,era,especially,essay,etc,etna,european,even,events,eventually,ever,every,everyone,everything,evil,evolution,exactly,example,excellent,except,exciting,existence,existing,expect,expected,experience,face,fact,fall,family,fan,far,fast,father,favorite,fbi,feel,feeling,feels,felt,female,fett,fiction,figure,finally,find,finds,finish,finished,fire,first,five,follow,...,put,quality,question,questions,quick,quickly,quite,rate,rates,rather,rating,re,read,reader,readers,reading,real,reality,really,reason,reasons,recommend,recommended,ref,relations,relationship,rest,return,review,reviews,revolution,rich,right,rights,romance,romantic,root,rosato,said,sands,save,saved,say,saying,says,scenes,scottoline,second,section,see,seem,seemed,seems,seen,self,sense,series,set,several,sex,she,ship,short,shreve,side,sight,simple,simply,since,sister,slave,slow,small,so,social,society,some,someone,something,sometimes,somewhat,soon,sort,source,south,sowell,space,speak,spoilers,stand,star,stars,start,started,starts,state,states,steam,still,stop,stories,story,straight,strong,struggle,style,subject,subscription,sure,suspense,system,take,takes,taking,tale,tassel,tell,tells,text,thanks,that,the,them,then,theory,there,these,they,thing,things,think,thinking,this,thomas,though,thought,thoughts,three,throughout,thus,tied,time,times,to,today,together,told,tony,too,took,town,trading,travel,trial,tribe,tried,trilogy,true,truly,try,trying,turn,turns,twist,two,type,ultimately,understand,understanding,unfortunately,unique,unlike,up,upon,us,use,used,usually,utf8,value,vampire,vampires,van,various,vatta,ve,version,very,viable,view,villain,volume,vong,wait,want,wanted,wants,war,wars,was,washington,way,ways,we,wealth,weight,well,went,what,when,whether,which,while,who,whole,whose,why,wife,will,wish,with,within,without,woman,women,wonderful,word,words,work,workers,working,works,world,worth,would,write,writer,writers,writes,writing,written,wrong,wrote,year,years,yes,yet,you,young,actual_ratings
929,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [17]:
new_train_df.head(3)

,19th,20th,abagnale,able,about,absolutely,academy,account,action,actual,actually,add,adventure,after,again,age,ago,all,allison,almost,alone,along,already,also,although,always,amazing,amazon,american,amount,and,anita,another,anyone,anything,are,argeneau,around,art,articles,as,at,attention,author,authors,available,away,back,bad,bank,based,battle,be,became,because,become,becomes,bed,before,begin,beginning,begins,behind,believable,believe,bennie,best,better,big,birthday,bit,bite,black,bliss,blood,boba,book,books,boring,bought,bourgeoisie,brennan,brief,bring,business,but,buy,by,call,came,can,cannot,capital,capitalism,capitalist,care,carnton,carrie,case,catch,cause,century,certainly,chance,chandler,change,chapter,chapters,character,characters,child,children,civil,class,classes,clear,clearly,cm_cr_arp_d_rvw_txt,collection,college,come,comes,coming,communism,communist,compelling,complete,completely,complex,con,concept,content,continue,control,copy,could,countries,country,couple,course,cover,created,crime,critique,culture,danger,dark,data,daughter,day,days,dead,deal,death,decided,definitely,descriptions,despite,detail,details,detective,developed,development,dialogue,did,diet,different,difficult,disappointed,division,do,doctor,done,down,dp,dr,due,earlier,early,easily,easy,economic,economics,edition,either,else,emily,end,ending,engels,engine,enjoy,enjoyable,enjoyed,enough,entertaining,entire,equal,era,especially,essay,etc,etna,european,even,events,eventually,ever,every,everyone,everything,evil,evolution,exactly,example,excellent,except,exciting,existence,existing,expect,expected,experience,face,fact,fall,family,fan,far,fast,father,favorite,fbi,feel,feeling,feels,felt,female,fett,fiction,figure,finally,find,finds,finish,finished,fire,first,five,follow,...,put,quality,question,questions,quick,quickly,quite,rate,rates,rather,rating,re,read,reader,readers,reading,real,reality,really,reason,reasons,recommend,recommended,ref,relations,relationship,rest,return,review,reviews,revolution,rich,right,rights,romance,romantic,root,rosato,said,sands,save,saved,say,saying,says,scenes,scottoline,second,section,see,seem,seemed,seems,seen,self,sense,series,set,several,sex,she,ship,short,shreve,side,sight,simple,simply,since,sister,slave,slow,small,so,social,society,some,someone,something,sometimes,somewhat,soon,sort,source,south,sowell,space,speak,spoilers,stand,star,stars,start,started,starts,state,states,steam,still,stop,stories,story,straight,strong,struggle,style,subject,subscription,sure,suspense,system,take,takes,taking,tale,tassel,tell,tells,text,thanks,that,the,them,then,theory,there,these,they,thing,things,think,thinking,this,thomas,though,thought,thoughts,three,throughout,thus,tied,time,times,to,today,together,told,tony,too,took,town,trading,travel,trial,tribe,tried,trilogy,true,truly,try,trying,turn,turns,twist,two,type,ultimately,understand,understanding,unfortunately,unique,unlike,up,upon,us,use,used,usually,utf8,value,vampire,vampires,van,various,vatta,ve,version,very,viable,view,villain,volume,vong,wait,want,wanted,wants,war,wars,was,washington,way,ways,we,wealth,weight,well,went,what,when,whether,which,while,who,whole,whose,why,wife,will,wish,with,within,without,woman,women,wonderful,word,words,work,workers,working,works,world,worth,would,write,writer,writers,writes,writing,written,wrong,wrote,year,years,yes,yet,you,young,actual_ratings
929,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

# 2. Model Building
## 2.1 Setting up the environment

In [19]:
numerical_features = list(features_df.columns)
#numerical_features

In [20]:
#the setup() function initializes the environment in pycaret and creates the transformation pipeline to prepare the data for modeling and deployment.

exp_regr = setup(data = new_train_df, target = 'actual_ratings',numeric_features=numerical_features,session_id=123) 

,Description,Value
0,session_id,123
1,Target,actual_ratings
2,Original Data,"(750, 801)"
3,Missing Values,False
4,Numeric Features,800
5,Categorical Features,0
6,Ordinal Features,False
7,High Cardinality Features,False
8,High Cardinality Method,None
9,Transformed Train Set,"(524, 787)"


## 2.2 Compare Model

In [21]:
top_model = compare_models()

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
br,Bayesian Ridge,0.8557,1.1876,1.0799,0.1211,0.2715,0.3519,0.4470
gbr,Gradient Boosting Regressor,0.8394,1.1884,1.0798,0.0990,0.2746,0.3318,0.3220
rf,Random Forest Regressor,0.8423,1.2025,1.0902,0.0756,0.2711,0.3355,1.0590
lightgbm,Light Gradient Boosting Machine,0.8581,1.2516,1.1088,0.0622,0.2728,0.3379,0.3000
et,Extra Trees Regressor,0.8506,1.2712,1.1211,0.0108,0.2773,0.3311,1.2950
en,Elastic Net,0.9202,1.4024,1.1745,-0.0355,0.2945,0.3898,0.0250
llar,Lasso Least Angle Regression,0.9224,1.4039,1.1754,-0.0378,0.2946,0.3900,0.0230
dummy,Dummy Regressor,0.9224,1.4039,1.1754,-0.0378,0.2946,0.3900,0.0250
lasso,Lasso Regression,0.9224,1.4039,1.1754,-0.0378,0.2946,0.3900,0.0240
knn,K Neighbors Regressor,0.8935,1.4649,1.1975,-0.0878,0.2990,0.3993,0.0460


In [23]:
br = create_model('br')

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,0.7138,0.6711,0.8192,-0.1274,0.1602,0.1683
1,1.0085,1.6552,1.2865,0.0417,0.3413,0.4941
2,0.7570,1.0059,1.0029,0.2376,0.2590,0.3173
3,0.8080,0.9030,0.9503,0.2635,0.2084,0.2393
4,0.8807,1.1182,1.0575,0.1684,0.2509,0.3047
5,0.9582,1.5058,1.2271,0.1817,0.3283,0.4613
6,0.7415,1.0440,1.0218,0.0723,0.2629,0.3099
7,0.9902,1.6135,1.2702,0.1166,0.3515,0.5160
8,0.9242,1.3915,1.1796,0.0694,0.3179,0.4339


In [24]:
top_model

BayesianRidge(alpha_1=1e-06, alpha_2=1e-06, alpha_init=None,
              compute_score=False, copy_X=True, fit_intercept=True,
              lambda_1=1e-06, lambda_2=1e-06, lambda_init=None, n_iter=300,
              normalize=False, tol=0.001, verbose=False)

## 2.3 Model Tuning

In [25]:
tuned_model = tune_model(top_model)

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,0.7138,0.6711,0.8192,-0.1275,0.1602,0.1683
1,1.0084,1.6549,1.2864,0.0419,0.3413,0.4940
2,0.7568,1.0056,1.0028,0.2378,0.2590,0.3173
3,0.8079,0.9029,0.9502,0.2636,0.2083,0.2392
4,0.8804,1.1179,1.0573,0.1687,0.2509,0.3046
5,0.9581,1.5057,1.2271,0.1818,0.3283,0.4613
6,0.7415,1.0439,1.0217,0.0724,0.2629,0.3099
7,0.9902,1.6131,1.2701,0.1169,0.3515,0.5160
8,0.9244,1.3918,1.1797,0.0692,0.3179,0.4339


In [ ]:
#plot_model(tuned_model)

In [ ]:
#plot_model(tuned_model, plot = 'error')

In [ ]:
#plot_model(tuned_model, plot='feature')

# 2.4 Model Evaluation and Tuning 

In [62]:
#evaluate_model(tuned_model);

In [28]:
predict_model(tuned_model);

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,Bayesian Ridge,0.8689,1.1796,1.0861,0.1468,0.2743,0.3451


In [30]:
unseen_predictions = predict_model(tuned_model, data=new_test_df)
unseen_predictions.head(20)

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,Bayesian Ridge,0.8999,1.2712,1.1275,0.0758,0.2848,0.3632


,19th,20th,abagnale,able,about,absolutely,academy,account,action,actual,...,wrong,wrote,year,years,yes,yet,you,young,actual_ratings,Label
241,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,4,3.618823
438,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,4.089549
744,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,3,3.993677
763,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,4,4.131086
691,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,5,3.868007
161,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,5,4.052128
334,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,1,0,0,3,3.576370
912,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,5,3.655642
523,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,5,4.200798
686,0,0,0,0,0,0,0,0,0,0,...,0,0,0,3,0,0,0,1,4,3.970483


In [31]:
save_model(tuned_model,"deployment_review_to_ratings_04142022")

Transformation Pipeline and Model Successfully Saved


(Pipeline(memory=None,
          steps=[('dtypes',
                  DataTypes_Auto_infer(categorical_features=[],
                                       display_types=True, features_todrop=[],
                                       id_columns=[], ml_usecase='regression',
                                       numerical_features=['19th', '20th',
                                                           'abagnale', 'able',
                                                           'about', 'absolutely',
                                                           'academy', 'account',
                                                           'action', 'actual',
                                                           'actually', 'add',
                                                           'adventure', 'after',
                                                           'again', 'age', 'ago',
                                                           'all', 'allison',
             